In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm

In [2]:
class Net(nn.Module):
    def __init__(self, input_dim=4, hid_dim=128, nlayers=2, out_dim=2):
        super(Net, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self._pi = nn.Linear(inp, out_dim)
        self._v = nn.Linear(inp, 1)
        self.model = nn.Sequential(*self.model)
    
    def forward(self, x):
        out = self.model(x)
        return self._pi(out), self._v(out)

In [3]:
class A2C:
    def __init__(self, model, optim, gamma, vl_coeff=0.5, el_coeff=0.01):
        self.model = model
        self.optim = optim
        self.gamma = gamma
        self.vl_coeff = vl_coeff
        self.el_coeff = el_coeff
    
    def learn(self, data):
        td_target = []
        R = 0
        if data['done'][-1]:
            final_s = torch.from_numpy(data['state'][-1]).float()
            with torch.no_grad():
                R = self.model(final_s)[1].item()
        
        for r in data['reward'][::-1]:
            R = r + self.gamma * R
            td_target.append(R)
        
        td_target = torch.Tensor(td_target[::-1])
        td_target = (td_target - td_target.mean()) / td_target.std()
        
        batch_state = torch.from_numpy(np.stack(data['state'][:-1])).float()
        logits, values = self.model(batch_state)
        value = values.squeeze(1)
        advantage = td_target - value
        action = data['action']
        prob = torch.softmax(logits, dim=1)
        
        entropy = dist.Categorical(prob).entropy()
        
        prob = prob[torch.arange(len(action)), action]
        
        
        self.optim.zero_grad()
        loss = -(torch.log(prob) * advantage.detach()).mean() + self.vl_coeff * ((advantage)**2).mean() - self.el_coeff * entropy.mean()
        loss.backward()
        self.optim.step()
    
    def sample_action(self, state):
        state = torch.from_numpy(state).float()
        prob = torch.softmax(self.model(state)[0], dim=0)
        m = dist.Categorical(prob)
        action = m.sample().item()
        return action

In [4]:
class Args:
    lr = 5e-4
    gamma = 0.98
    max_episodes = 1000

In [5]:
args = Args

In [ ]:
env = gym.make('CartPole-v1')
lr = args.lr
gamma = args.gamma
max_episodes = args.max_episodes

n_obs = env.observation_space.shape[0]
n_act = env.action_space.n
model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
optimizer = optim.Adam(model.parameters(), lr=lr)
policy = A2C(model, optimizer, gamma)

In [ ]:
all_score_list = []
for k in range(10):
    env = gym.make('CartPole-v1')
    lr = args.lr
    gamma = args.gamma
    max_episodes = args.max_episodes

    n_obs = env.observation_space.shape[0]
    n_act = env.action_space.n
    model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    policy = A2C(model, optimizer, gamma)
    
    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        s = env.reset()
        data_buffer = {'state': [s], 'reward': [], 'action': [], 'done': []}
        done = False
        while not done:
            a = policy.sample_action(s)
            next_s, r, done, info = env.step(a)
            data_buffer['reward'].append(r)
            data_buffer['action'].append(a)
            data_buffer['done'].append(done)
            data_buffer['state'].append(next_s)
            s = next_s

        policy.learn(data_buffer)
        score_list.append(sum(data_buffer['reward']))
    
    all_score_list.append(score_list)